In [1]:
import pandas as pd
import warnings
warnings.filterwarnings("ignore")

# Loading the datasets
accounts = pd.read_csv("../data/raw/ravenstack_accounts.csv")
subscriptions = pd.read_csv("../data/raw/ravenstack_subscriptions.csv")
churn = pd.read_csv("../data/raw/ravenstack_churn_events.csv")

# Convert date columns
subscriptions['start_date'] = pd.to_datetime(subscriptions['start_date'], dayfirst=True)
subscriptions['end_date'] = pd.to_datetime(subscriptions['end_date'], dayfirst=True)
churn['churn_date'] = pd.to_datetime(churn['churn_date'], dayfirst=True)

print("Completed!")

Completed!


In [2]:
# Creating monthly range
'''months = pd.date_range(
    start="2023-01-01",
    end="2024-12-01",
    freq="MS"
)

# Creating backbone (account × month)
backbone = pd.MultiIndex.from_product(
    [accounts['account_id'].unique(), months],
    names=["account_id", "snapshot_month"]
).to_frame(index=False)

print("Backbone columns:", backbone.columns.tolist())
print("Backbone rows:", len(backbone))'''

global_start = subscriptions['start_date'].min().replace(day=1)
global_end = churn['churn_date'].max().replace(day=1)

months = pd.date_range(start=global_start, end=global_end, freq="MS")

print("Month range:", global_start, "-", global_end)

Month range: 2023-01-01 00:00:00 - 2024-12-01 00:00:00


In [3]:
# creating backbone (account × month)
backbone = pd.MultiIndex.from_product([accounts['account_id'].unique(), months],
                                       names=["account_id", "snapshot_month"]).to_frame(index=False)

In [4]:
# building subscription summary
subscription_summary = subscriptions.groupby(
    'account_id', as_index=False
).agg(
    start_date=('start_date', 'min'),
    end_date=('end_date', 'max')
)

print("Subscription summary columns:", subscription_summary.columns.tolist())
print("Subscription summary rows:", len(subscription_summary))

Subscription summary columns: ['account_id', 'start_date', 'end_date']
Subscription summary rows: 500


In [5]:
# merging subscription summary with backbone
backbone = backbone.merge(
    subscription_summary,
    on="account_id",
    how="left",
    validate="many_to_one"
)

print("After merge columns:", backbone.columns.tolist())
print("After merge rows:", len(backbone))

After merge columns: ['account_id', 'snapshot_month', 'start_date', 'end_date']
After merge rows: 12000


In [6]:
# Keeping only months which are in active subscription period
backbone = backbone[
    (backbone['snapshot_month'] >= backbone['start_date']) &
    (
        backbone['end_date'].isna() |
        (backbone['snapshot_month'] <= backbone['end_date'])
    )]

print("Rows after filtering:", len(backbone))

Rows after filtering: 4109


In [7]:
# merging churn date into backbone
churn = pd.read_csv("../data/raw/ravenstack_churn_events.csv")
churn['churn_date'] = pd.to_datetime(churn['churn_date'], dayfirst=True)

# keeping only needed columns
churn = churn[['account_id', 'churn_date']]

# merge with backbone
backbone = backbone.merge(churn, on='account_id', how='left')

print('Final backbone columns:', backbone.columns.tolist())

Final backbone columns: ['account_id', 'snapshot_month', 'start_date', 'end_date', 'churn_date']


### Creating labels for Early Churn 
- Label 1: if churn within 2 months
- Label 0: else

In [8]:
# Defining prediction window (2 months)
backbone['label'] = (
    (backbone['churn_date'].notna()) &
    (backbone['churn_date'] > backbone['snapshot_month']) &
    (backbone['churn_date'] <= backbone['snapshot_month'] + pd.DateOffset(months=2))
).astype(int)

print("Label distribution:")
print(backbone['label'].value_counts())

'''# Defining effective_end_date
backbone['effective_end_date'] = backbone['churn_date'].combine_first(backbone['end_date'])

# Now re-filtering the backbone
backbone = backbone[
    (backbone['snapshot_month'] >= backbone['start_date']) &
    (
        backbone['effective_end_date'].isna() |
        (backbone['snapshot_month'] <= backbone['effective_end_date'])
    )]'''

Label distribution:
label
0    5361
1     757
Name: count, dtype: int64


"# Defining effective_end_date\nbackbone['effective_end_date'] = backbone['churn_date'].combine_first(backbone['end_date'])\n\n# Now re-filtering the backbone\nbackbone = backbone[\n    (backbone['snapshot_month'] >= backbone['start_date']) &\n    (\n        backbone['effective_end_date'].isna() |\n        (backbone['snapshot_month'] <= backbone['effective_end_date'])\n    )]"

#### The following code was just to validate our labels. These are not the part of the pipeline.

In [ ]:
# we want to know does every churn account have atleast one positive label in backbone?
'''positive_accounts = backbone[backbone['label'] == 1]['account_id'].nunique()
total_churned_accounts = churn['account_id'].nunique()

print("Accounts with positive label:", positive_accounts)
print("Total churned accounts:", total_churned_accounts)'''

Accounts with positive label: 282
Total churned accounts: 352


In [ ]:
# checking if accounts have too many positive labels
'''backbone[backbone['label'] == 1].groupby('account_id').size().describe()'''

count    282.000000
mean       2.684397
std        1.445217
min        1.000000
25%        2.000000
50%        2.000000
75%        4.000000
max        8.000000
dtype: float64

In [ ]:
# Find churned accounts with zero positive label
'''churned_accounts = set(churn['account_id'])
positive_accounts = set(backbone[backbone['label'] == 1]['account_id'])

missing_accounts = churned_accounts - positive_accounts

print("Missing churn accounts:", len(missing_accounts))'''

Missing churn accounts: 70


In [ ]:
list(missing_accounts)[:10]

['A-09540c',
 'A-439b2f',
 'A-463db0',
 'A-558c72',
 'A-417d2f',
 'A-342303',
 'A-0f7d77',
 'A-20736a',
 'A-39ddf4',
 'A-7df7a7']

In [ ]:
'''acc = list(missing_accounts)[0]

print(backbone[backbone['account_id'] == acc][
    ['snapshot_month','start_date','churn_date']
])'''

Empty DataFrame
Columns: [snapshot_month, start_date, churn_date]
Index: []


In [ ]:
'''missing_accounts = list(missing_accounts)

check = subscriptions[subscriptions['account_id'].isin(missing_accounts)]

print(check[['account_id','start_date','end_date']].head())
print("Min start_date:", check['start_date'].min())
print("Max start_date:", check['start_date'].max())
'''

   account_id start_date   end_date
6    A-417d2f 2023-12-30        NaT
7    A-5f2961 2024-12-23        NaT
16   A-86902e 2023-07-21 2024-08-03
23   A-aa9511 2024-02-09        NaT
36   A-40a557 2024-02-12        NaT
Min start_date: 2023-02-05 00:00:00
Max start_date: 2024-12-31 00:00:00
